# Testing and Benchmarking

We load the best weights we've found, put them in a model, generate predictions.

In [1]:
#Load best model weights
from keras.models import load_model
model = load_model('tmp/weights-32b-106-80-19_hsv.hdf5')

C:\Users\TWright\Anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Unlike training we resize the images online here, it takes about 10 minutes.

In [2]:
dim = 106

In [3]:
import glob
import numpy as np
from keras.preprocessing.image import img_to_array
from skimage.transform import resize
from skimage.io import imread
from skimage.color import rgb2hsv

def fetch_test_images():
    while 1:
        for fname in test_files:
            image = imread(fname).T
            image = img_to_array(image)
            test = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            yield (test)

test_files = glob.glob('../data/images_test_rev1_106/*.jpg')
test_files[0:5]

In [5]:
import matplotlib.pylab as plt
%matplotlib inline

sample_img = preprocess_image(test_files[0])
plt.imshow(sample_img.T)
plt.show()

NameError: name 'preprocess_image' is not defined

Generate predictions.

In [6]:
predictions = model.predict_generator(fetch_test_images(),
                        steps=len(test_files),
                        max_queue_size=32,)
predictions.shape # assert (79975, 37)

(79975, 37)

You can visually verify predictions, first three numbers should add up to 1.

In [7]:
predictions[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 7.5201491e-37,
       1.9390095e-24, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00], dtype=float32)

Record all results into a csv file with the id and the predictions. The CSV file will be about 40MB big. We only load the all_zeros.csv file for its header.

In [9]:
from os import path

header = open('../data/all_zeros_benchmark.csv','r').readlines()[0]

with open('submission_1.csv','w') as outfile:
    outfile.write(header)
    for i, fname in enumerate(test_files):
        fname = test_files[i]
        file_id = path.split(fname)[-1] \
                      .split('.')[0]
        pred = predictions[i]
        outline = file_id + "," + ",".join([str(x) for x in pred])
        outfile.write(outline + "\n")